In [3]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver  


agent = create_agent(
    "gpt-3.5-turbo",
    checkpointer=InMemorySaver(),  
)

while True:
    msg = input("Enter msg")
    output = agent.invoke(
        {"messages": [{"role": "user", "content": msg}]},
        {"configurable": {"thread_id": "1"}},  
    )

    print(output)

    if msg in ["exit", "quit"]:
        break


{'messages': [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}, id='2227fd84-c064-4229-abbc-f669bce5bea9'), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CXjHpEvXaSxYjUhHlcJysHawXkdQv', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--506f5616-62c6-4a19-a508-503be10c71fb-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}
{'messages': [HumanM

## using Workflow

In [10]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict, Annotated
from langchain_openai import ChatOpenAI


In [5]:
class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]


In [ ]:
# Initialize model
model = ChatOpenAI(model="gpt-3.5-turbo")

# Define node function
def chat_node(state: State):
    # model returns a new message
    response = model.invoke(state["messages"])
    return {"messages": [response]}


In [22]:
graph_builder = StateGraph(State)

# Add one node
graph_builder.add_node("chat", chat_node)

# Define flow: START → chat → END
graph_builder.add_edge(START, "chat")
graph_builder.add_edge("chat", END)


In [23]:
checkpointer = InMemorySaver()
graph = graph_builder.compile(checkpointer=checkpointer)


In [24]:
while True:
    msg = input("You: ")
    print("msg:", msg)

    if msg.lower() in ["exit", "quit"]:
        print("Goodbye!")
        break

    # Build state for this run
    state = {"messages": [HumanMessage(content=msg)]}

    # Config carries thread ID (memory key)
    config = {"configurable": {"thread_id": "1"}}

    result = graph.invoke(state, config=config)
    ai_msg = result["messages"][-1].content

    print("AI:", ai_msg)


msg: hello
state messages: [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}, id='f0b5fc37-7208-4f06-8a0b-84831c1c4526')]


InvalidUpdateError: Expected dict, got [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}, id='f0b5fc37-7208-4f06-8a0b-84831c1c4526')]
For troubleshooting, visit: https://docs.langchain.com/oss/python/langgraph/errors/INVALID_GRAPH_NODE_RETURN_VALUE